<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/Web_image_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**画像のスクレイピング**

準備：
- Microsoft AZUREに登録

    https://learn.microsoft.com/ja-jp/azure/cognitive-services/bing-web-search/

- 左のタブ → リソースの作成 → Bing Search v7を取得

- ダッシュボード → キーとエンドポイントからキーを取得する


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

bing_api_key = key[13]

In [ ]:
from requests import exceptions
import argparse
import requests
import cv2
import os

API_KEY = f"{bing_api_key}"
MAX_RESULTS = 20
GROUP_SIZE = 20

# 取得したエンドポイントURL
URL = "https://api.bing.microsoft.com/v7.0/images/search"
OUTPUT = '/content/save_dir'

if not os.path.isdir(OUTPUT):
    os.mkdir(OUTPUT)

EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])

term = 'cat'
headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType":"Photo", "color":"ColorOnly"}

# make the search
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

# grab the results from the search, including the total number of
# estimated results returned by the Bing API
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults,
    term))

# initialize the total number of images downloaded thus far
total = 0

# loop over the estimated number of results in `GROUP_SIZE` groups
for offset in range(0, estNumResults, GROUP_SIZE):
    # update the search parameters using the current offset, then
    # make the request to fetch the results
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    # loop over the results
    for v in results["value"]:
        # try to download the image
        try:
            # make a request to download the image
            print("[INFO] fetching: {}".format(v["contentUrl"]))
            r = requests.get(v["contentUrl"], timeout=30)

            # build the path to the output image
            print
            ext = v["contentUrl"][v["contentUrl"].rfind("."):v["contentUrl"].rfind("?") if v["contentUrl"].rfind("?") > 0 else None]
            p = os.path.sep.join([OUTPUT, "{}{}".format(
                str(total).zfill(8), ext)])

            # write the image to disk
            f = open(p, "wb")
            f.write(r.content)
            f.close()

        # catch any errors that would not unable us to download the
        # image
        except Exception as e:
            # check to see if our exception is in our list of
            # exceptions to check for
            if type(e) in EXCEPTIONS:
                print("[INFO] skipping: {}".format(v["contentUrl"]))
                continue
        # try to load the image from disk
        image = cv2.imread(p)

        # if the image is `None` then we could not properly load the
        # image from disk (so it should be ignored)
        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            continue

        # update the counter
        total += 1

#**Chromedriverを用いる方法**

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium==4.1.0 #新しいバージョンだとエラーが出るので旧バージョンにする

In [60]:
!apt-get update
!apt-get install -y firefox

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib.request
import time

# Firefoxのバイナリーパスを取得する
!which firefox

binary = '/usr/bin/firefox'
options = webdriver.FirefoxOptions()
options.binary_location = binary
options.add_argument('--no-sandbox')

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList",2)
profile.set_preference("browser.download.manager.showWhenStarting",False)
profile.set_preference("browser.download.dir", "/content/drive/MyDrive/")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "image/jpeg")

# Firefoxドライバーを設定する
driver = webdriver.Firefox(executable_path='/usr/bin/geckodriver', firefox_profile=profile, options=options)

driver.get("https://www.google.co.jp/imghp")
elem = driver.find_element_by_name("q")
elem.send_keys("キャットフード")
elem.submit()

SCROLL_PAUSE_TIME = 2

# スクロールして画像を追加読み込みする
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # スクロールダウン
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 画像読み込み待ち
    time.sleep(SCROLL_PAUSE_TIME)
    # 画面の高さが変わらない場合終了
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 画像のリンク取得
image_links = []
for i in driver.find_elements_by_css_selector(".rg_i"):
    link = i.get_attribute('src')
    if link is not None and link.startswith('http'):
        image_links.append(link)

# 画像ダウンロード
for i, link in enumerate(image_links):
    filename = f"{i}.jpg"
    urllib.request.urlretrieve(link, filename)
    print(f"Downloaded {filename}")

driver.close()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

<ipython-input-60-9c353b1645f1>:17: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
<ipython-input-60-9c353b1645f1>:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='/usr/bin/geckodriver', firefox_profile=profile, options=options)
<ipython-input-60-9c353b1645f1>:24: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(executable_path='/usr/bin/geckodriver', firefox_profile=profile, options=options)


InvalidArgumentException: ignored

In [ ]:
!apt-get update
!apt-get install -y wget curl unzip
!apt-get install -y libglib2.0-0 libnss3 libgconf-2-4 libfontconfig1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!google-chrome-stable --version

!wget https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver

!pip install selenium


In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import urllib.request
import time
import os

# ダウンロードする画像の検索キーワードを指定
searchterm = "cat"

# ダウンロードする枚数を指定
num_images = 5

# chromedriverをColabにダウンロード
!apt update
!apt install -y wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install
!apt install -y unzip
!wget https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
os.environ["webdriver.chrome.driver"] = "/content/chromedriver"

# Chromeを起動
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-extensions")
options.add_argument("--incognito")
options.add_argument("--disable-plugins-discovery")
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-gpu")
options.add_argument("--disable-infobars")
driver = webdriver.Chrome(options=options)

# Google画像検索を開く
driver.get("https://www.google.com/search?q=" + searchterm + "&source=lnms&tbm=isch")

# スクロールダウンして画像を取得
i = 0
while i < num_images:
    driver.execute_script("window.scrollBy(0,10000)")
    try:
        driver.find_element_by_css_selector(".mye4qd").click()
        time.sleep(2)
    except:
        pass
    i += 1

# 画像をダウンロード
images = driver.find_elements_by_css_selector(".rg_i")
image_links = set()
for image in images:
    image_link = image.get_attribute("src")
    if image_link.endswith(".jpg") or image_link.endswith(".png"):
        image_links.add(image_link)

# ダウンロードした画像を保存
try:
    os.makedirs(searchterm)
except FileExistsError:
    pass
for i, image_link in enumerate(image_links):
    if i >= num_images:
        break
    image_path = os.path.join(searchterm, f"{searchterm}_{i}.jpg")
    urllib.request.urlretrieve(image_link, image_path)
    
# Chromeを終了
driver.quit()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:15 http://ppa.launchpad

WebDriverException: ignored

In [ ]:
QUERY = 'パイナップル　フリー'                        # 検索ワード
SAVE_DIR = '/content/pine'                        # 出力フォルダへのパス（フォルダがない場合は自動生成する）
FILE_NAME = 'pine_'                          # ファイル名（ファイル名の後ろに０からの連番と拡張子が付く）


import requests
!pip install selenium --q
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
 
tm_start = time.time()            #処理時間計測用
dt_now = datetime.datetime.now()  # 現在日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M')
print(dt_date_str)
 
# QUERY = 'りんご　フリー'                        # 検索ワード
LIMIT_DL_NUM = 10                          # ダウンロード数の上限
# SAVE_DIR = '/content/apple'                        # 出力フォルダへのパス（フォルダがない場合は自動生成する）
# FILE_NAME = 'strawberry_'                          # ファイル名（ファイル名の後ろに０からの連番と拡張子が付く）
TIMEOUT = 60                                # 要素検索のタイムアウト（秒）
ACCESS_WAIT = 1                             # アクセスする間隔（秒）
RETRY_NUM = 3                               # リトライ回数（クリック、requests）
DRIVER_PATH = '/usr/lib/chromium-browser/chromedriver'       # chromedriver.exeへのパス
 
# Chromeをヘッドレスモードで起動
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
driver = webdriver.Chrome(DRIVER_PATH, options=options)
 
# タイムアウト設定
driver.implicitly_wait(TIMEOUT)
 
tm_driver = time.time()
print('WebDriver起動完了', f'{tm_driver - tm_start:.1f}s')
 
# Google画像検索ページを取得
url = f'https://www.google.com/search?q={QUERY}&tbm=isch'
driver.get(url)
 
tm_geturl = time.time()
print('Google画像検索ページ取得', f'{tm_geturl - tm_driver:.1f}s')
 
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
 
count = len(tmb_alts) - tmb_alts.count('')
print(count)
 
while count < LIMIT_DL_NUM:
    # ページの一番下へスクロールして新しいサムネイル画像を表示させる
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
 
    # サムネイル画像取得
    tmb_elems = driver.find_elements_by_css_selector('#islmp img')
    tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
 
    count = len(tmb_alts) - tmb_alts.count('')
    print(count)  
 
# サムネイル画像をクリックすると表示される領域を取得
imgframe_elem = driver.find_element_by_id('islsp')
 
# 出力フォルダ作成
os.makedirs(SAVE_DIR, exist_ok=True)
 
# HTTPヘッダ作成
HTTP_HEADERS = {'User-Agent': driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)           
            
# ダウンロード対象のファイル拡張子
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.gif')
 
# 拡張子を取得
def get_extension(url):
    url_lower = url.lower()
    for img_ext in IMG_EXTS:
        if img_ext in url_lower:
            extension = '.jpg' if img_ext == '.jpeg' else img_ext
            break
    else:
        extension = ''
    return extension
 
# urlの画像を取得しファイルへ書き込む
def download_image(url, path, loop):
    result = False
    for i in range(loop):
        try:
            r = requests.get(url, headers=HTTP_HEADERS, stream=True, timeout=10)
            r.raise_for_status()
            with open(path, 'wb') as f:
                f.write(r.content)

        except requests.exceptions.SSLError:
            print('***** SSL エラー')
            break  # リトライしない
        except requests.exceptions.RequestException as e:
            print(f'***** requests エラー({e}): {i + 1}/{RETRY_NUM}')
            time.sleep(1)
        else:
            result = True
            break  # try成功
    return result
 
tm_thumbnails = time.time()
print('サムネイル画像取得', f'{tm_thumbnails - tm_geturl:.1f}s')
 
# ダウンロード
EXCLUSION_URL = 'https://lh3.googleusercontent.com/'  # 除外対象url
count = 0
url_list = []
for tmb_elem, tmb_alt in zip(tmb_elems, tmb_alts):
     
    if tmb_alt == '':
        continue
 
    print(f'{count}: {tmb_alt}')
 
    for i in range(RETRY_NUM):
        try:
            # サムネイル画像をクリック
            tmb_elem.click()
        except ElementClickInterceptedException:
            print(f'***** click エラー: {i + 1}/{RETRY_NUM}')
            driver.execute_script('arguments[0].scrollIntoView(true);', tmb_elem)
            time.sleep(1)
        else:
            break  # try成功
    else:
        print('***** キャンセル')
        continue  # リトライ失敗
         
    # アクセス負荷軽減用のウェイト
    time.sleep(ACCESS_WAIT)
     
    alt = tmb_alt.replace("'", "\'")
    try:
        img_elem = imgframe_elem.find_element_by_css_selector(f'img[alt=\'{alt}\']')
    except NoSuchElementException:
        print('***** img要素検索エラー')
        print('***** キャンセル')
        continue
 
    # url取得
    tmb_url = tmb_elem.get_attribute('src')  # サムネイル画像のsrc属性値
 
    for i in range(RETRY_NUM):
        url = img_elem.get_attribute('src')
        if EXCLUSION_URL in url:
            print('***** 除外対象url')
            url = ''
            break
        elif url == tmb_url:  # src属性値が遷移するまでリトライ
            print(f'***** urlチェック: {i + 1}/{RETRY_NUM}')
            time.sleep(1)
            url = ''
        else:
            break
 
    if url == '':
        print('***** キャンセル')
        continue
 
    # 画像を取得しファイルへ保存
    ext = get_extension(url)
    if ext == '':
        print(f'***** urlに拡張子が含まれていないのでキャンセル')
        print(f'{url}')
        continue
 
    filename = f'{FILE_NAME}{count}{ext}'
    path = SAVE_DIR + '/' + filename
    result = download_image(url, path, RETRY_NUM)
    if result == False:
        print('***** キャンセル')
        continue
    url_list.append(f'{filename}: {url}')
 
    # ダウンロード数の更新と終了判定
    count += 1
    if count >= LIMIT_DL_NUM:
        break
 
tm_end = time.time()
print('ダウンロード', f'{tm_end - tm_thumbnails:.1f}s')
print('------------------------------------')
total = tm_end - tm_start
total_str = f'トータル時間: {total:.1f}s({total/60:.2f}min)'
count_str = f'ダウンロード数: {count}'
print(total_str)
print(count_str)
 
# # urlをファイルへ保存
# path = SAVE_DIR + '/url/' + '_url.txt'
# with open(path, 'w', encoding='utf-8') as f:
#     f.write(dt_date_str + '\n')
#     f.write(total_str + '\n')
#     f.write(count_str + '\n')
#     f.write('\n'.join(url_list))
 
driver.quit()